<a href="https://colab.research.google.com/github/sbogde/pandamonium/blob/main/dmi_fifa_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFA 21 Player Dataset

**Data Source:** This dataset comes from [Kaggle's FIFA 21 complete player dataset](https://www.kaggle.com/datasets/stefanoleone992/fifa-21-complete-player-dataset/data).


The dataset contains 100+ attributes like player positions, with the role in the club and in the national team, player attributes with statistics as Attacking, Skills, Defense, Mentality, GK Skills, etc., and player personal data like Nationality, Club, DateOfBirth, Wage, Salary, etc..


In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import folium
from folium.plugins import MarkerCluster

from geopy.geocoders import Nominatim

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# Environment-agnostic data loading: Dynamically configure file path for Colab/local compatibility

if 'google.colab' in str(get_ipython()):
    from google.colab import drive, data_table
    drive.mount('/content/drive', force_remount=True)
    file_path = '/content/drive/My Drive/Colab Notebooks/data-mining/data/players_21.csv'
    data_table.enable_dataframe_formatter()
else:
    file_path = './data/players_21.csv'  # Local path

Mounted at /content/drive


In [3]:
df = pd.read_csv(file_path)

In [4]:
df.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club_name,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,33,1987-06-24,170,72,Argentina,FC Barcelona,...,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,35,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,200389,https://sofifa.com/player/200389/jan-oblak/210002,J. Oblak,Jan Oblak,27,1993-01-07,188,87,Slovenia,Atlético Madrid,...,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,31,1988-08-21,184,80,Poland,FC Bayern München,...,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,28,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3


In [5]:
print("Dataset shape:", df.shape)

Dataset shape: (18944, 106)


In [6]:
df.isnull().sum()

,0
sofifa_id,0
player_url,0
short_name,0
long_name,0
age,0
...,...
lb,0
lcb,0
cb,0
rcb,0


In [7]:
# Define key performance-related columns for filtering and analysis
key_columns = [
    'overall', 'potential', 'value_eur', 'wage_eur', 'player_positions',
    'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic'
]
df[key_columns].isnull().sum()

,0
overall,0
potential,0
value_eur,0
wage_eur,0
player_positions,0
pace,2083
shooting,2083
passing,2083
dribbling,2083
defending,2083


In [8]:
# Remove goalkeepers for classification and clustering tasks
df = df[~df['player_positions'].str.contains('GK')]

In [9]:
# Drop rows that contain missing values in any of the key analysis columns.
# This creates our clean, final dataset (df_cleaned) which will be used
# for all subsequent modelling and exploration tasks. Goalkeepers have
# already been removed, and the data is now limited to outfield players
# with complete data across key metrics like pace, shooting, and value.

# df_cleaned = df.dropna(subset=key_columns)
df_cleaned = df.dropna(subset=key_columns).copy()

df_cleaned.shape

(16860, 106)

In [10]:
# Feature engineering: Calculate Body Mass Index (BMI) and extract primary player position
df_cleaned['BMI'] = df_cleaned['weight_kg'] / ((df_cleaned['height_cm'] / 100) ** 2)
df_cleaned['main_position'] = df_cleaned['player_positions'].apply(lambda x: x.split(',')[0].strip())
df_cleaned.shape

(16860, 108)

In [11]:
# Encode player positions as numerical labels for classification tasks
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cleaned['position_label'] = le.fit_transform(df_cleaned['main_position'])
df_cleaned.shape

(16860, 109)

In [12]:
# Show mapping of labels
position_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Position label mapping:\n", position_mapping)

Position label mapping:
 {'CAM': np.int64(0), 'CB': np.int64(1), 'CDM': np.int64(2), 'CF': np.int64(3), 'CM': np.int64(4), 'LB': np.int64(5), 'LM': np.int64(6), 'LW': np.int64(7), 'LWB': np.int64(8), 'RB': np.int64(9), 'RM': np.int64(10), 'RW': np.int64(11), 'RWB': np.int64(12), 'ST': np.int64(13)}


In [13]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV


In [ ]:
# Define features and target
features = ['age', 'height_cm', 'weight_kg', 'pace', 'shooting', 'passing',
            'dribbling', 'defending', 'physic', 'BMI']
X = df_cleaned[features]
y = df_cleaned['main_position']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model and grid
clf = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': range(5, 20),
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Grid search
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy (CV):", grid_search.best_score_)

# Best parameters: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 15}
# Best accuracy (CV): 0.5383318862102222

In [ ]:

# features = ['age', 'height_cm', 'weight_kg', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
# df = df.dropna(subset=features + ['player_positions'])  # Clean missing values

# X = df[features]
# y = df['player_positions'].apply(lambda x: x.split(",")[0])  # Primary position only

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the model
# clf = DecisionTreeClassifier(random_state=42)

# # Define parameter grid
# param_grid = {
#     'max_depth': range(5, 20),
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 5, 10],
#     'criterion': ['gini', 'entropy']
# }

# # Grid Search
# grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_train, y_train)

# print("Best parameters:", grid_search.best_params_)
# print("Best accuracy:", grid_search.best_score_)

# Best parameters: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 10, 'min_samples_split': 2}
# Best accuracy: 0.5461150447756107

In [ ]:


# Select features and target
features = ['age', 'height_cm', 'weight_kg', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']
df = df.dropna(subset=features + ['player_positions'])  # Clean missing values

X = df[features]
y = df['player_positions'].apply(lambda x: x.split(",")[0])  # Primary position only

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Classifier
# clf = DecisionTreeClassifier(max_depth=4, random_state=42) # 0.4533
# clf = DecisionTreeClassifier(max_depth=5, random_state=42) # 0.4909
# clf = DecisionTreeClassifier(max_depth=6, random_state=42) # 0.5161
# clf = DecisionTreeClassifier(max_depth=7, random_state=42) # 0.5375
# clf = DecisionTreeClassifier(max_depth=11, random_state=42) # 0.5383
# clf = DecisionTreeClassifier(max_depth=15, random_state=42) # 0.4951

# 0.5517
clf = DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_leaf=10, min_samples_split=2, random_state=42) # 0.4951

clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:", classification_report(y_test, y_pred))


In [ ]:
df = pd.read_csv(file_path)
print("Dataset shape:", df.shape)

# Select numeric features for clustering
features = ['age', 'overall', 'potential', 'value_eur', 'wage_eur']
df = df.dropna(subset=features)

X = df[features]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Visualize clusters
plt.scatter(df['overall'], df['value_eur'], c=df['Cluster'], cmap='viridis')
plt.xlabel('Overall Rating')
plt.ylabel('Market Value (EUR)')
plt.title('Player Clusters by Rating and Market Value')
plt.show()
